# MapReduce in Word Counting

In [1]:
import urllib, string, random
from nltk import word_tokenize
from collections import Counter
from operator import itemgetter
from collections import defaultdict
from functools import partial
from itertools import product
import numpy as np

### 0. Data: Alice's Adventure

In [2]:
firehose = urllib.urlopen(url="http://www.gutenberg.org/cache/epub/11/pg11.txt")
documents = firehose.readlines()

### 1. Old Technique: Brute Force

In [3]:
def word_count_old(documents):
    return Counter(word for document in documents for word in word_tokenize(document))

In [4]:
%%time
countOld = word_count_old(documents)
countOld = sorted([(word,count) for word,count in countOld.iteritems()], key=itemgetter(1), reverse=True)

CPU times: user 622 ms, sys: 10.5 ms, total: 633 ms
Wall time: 630 ms


In [5]:
countOld[:10]

[(',', 2565),
 ('the', 1675),
 ("'", 1130),
 ('.', 1101),
 ('and', 824),
 ('to', 791),
 ('a', 669),
 ('of', 602),
 ('it', 525),
 ('she', 506)]

### 2. MapReduce

In [6]:
class MapReduceWC:
    # NB: _.. methods are module-private, not imported when import *.py
    # NB: __.. methods are class-private, not accessible.

    def __init__(self, documents):
        self.documents = documents
        self.collector = defaultdict(list)
    
    def __wc_mapper(self, document):
        # for each word in the document, lazy-return (word,1).
        for word in word_tokenize(document):
            yield (word,1)
            
    def __wc_reducer(self, word, counts):
        # sum up the counts for a word.
        yield (word, sum(counts))
        
    def word_count(self):
        for document in self.documents:
            for word,count in self.__wc_mapper(document):
                self.collector[word].append(count)
        return [output
                for word,counts in self.collector.iteritems()
                for output in self.__wc_reducer(word,counts)]

In [7]:
%%time
mrwc = MapReduceWC(documents)
countNew = mrwc.word_count()

CPU times: user 577 ms, sys: 37.3 ms, total: 614 ms
Wall time: 588 ms


In [8]:
countNew = sorted(countNew, key=itemgetter(1), reverse=True)
countNew[:10]

[(',', 2565),
 ('the', 1675),
 ("'", 1130),
 ('.', 1101),
 ('and', 824),
 ('to', 791),
 ('a', 669),
 ('of', 602),
 ('it', 525),
 ('she', 506)]

### 4. Additional Benchmarking with Brown

**NB: No performance difference with 1 machine. However MapReduce allows distributed computing.**

In [9]:
from nltk.corpus import brown
brownSents = [' '.join(sent) for sent in brown.sents()]

In [10]:
%%time
# OLD WORD COUNT METHOD
countOld = word_count_old(brownSents)
countOld = sorted([(word,count) for word,count in countOld.iteritems()], key=itemgetter(1), reverse=True)
print countOld[:10]

[(u'the', 62713), (u',', 58336), (u'.', 50270), (u'of', 36080), (u'and', 27915), (u'to', 25732), (u'a', 21888), (u'in', 19540), (u'that', 10328), (u'is', 10100)]
CPU times: user 9.98 s, sys: 122 ms, total: 10.1 s
Wall time: 10 s


In [11]:
%%time
# NEW WORD COUNT METHOD
mrwc = MapReduceWC(brownSents)
countNew = mrwc.word_count()
countNew = sorted(countNew, key=itemgetter(1), reverse=True)
print countNew[:10]

[(u'the', 62713), (u',', 58336), (u'.', 50270), (u'of', 36080), (u'and', 27915), (u'to', 25732), (u'a', 21888), (u'in', 19540), (u'that', 10328), (u'is', 10100)]
CPU times: user 9.97 s, sys: 117 ms, total: 10.1 s
Wall time: 10 s


# MapReduce in General

In [12]:
# GENERALIZED MAPREDUCE
def map_reduce(inputs, mapper, reducer):
    collector = defaultdict(list)
    for inpt in inputs:
        for key,value in mapper(inpt):
            collector[key].append(value)
    return [output
            for key,values in collector.iteritems()
            for output in reducer(key,values)]

In [13]:
# GENERALIZED REDUCER
def reduce_values_using(aggregation_fn, key, values):
    yield (key, aggregation_fn(values))
def values_reducer(aggregation_fn):
    # turns a (values->output) aggregation function into a reducer: (key,values)->(key,output).
    return partial(reduce_values_using, aggregation_fn)

In [14]:
# CREATEING SPECIALIZED REDUCERS
sum_reducer = values_reducer(sum)
max_reducer = values_reducer(max)
min_reducer = values_reducer(min)
count_distinct_reduer = values_reducer(lambda values: len(set(values)))

# Application: Matrix Multiplication

** a. Matrix Representation**

* Instead of storing (sparse) matrices using list of lists, only store non-zero cells with their indices indicated (e.g. $("A", i, j, value)$ for $A_{ij}$ if $A_{ij}\neq 0$). 

** b. Efficient Computation Using Indices**

* For $C = A\cdot B$ a cell in matrix $C$, $C_{ik}$ is only related to $A_{ij}$ and $B_{jk}$.
* In the following graph, $i$ and $k$ locate a cell in $C$ (i.e. $C_{ik}$), the column/row index of $A$/$B$, $j$, is used to indicate that these two cells "meet" in the computation.
* Therefore, for a pair of element representation $("A", i, j, value1)$ and $("B", j, k, value2)$, we know extract $(i,k)$ to locate the cell $C_{ik}$, and use $j$ to indicate that these two cells meet in computation.
* **NB:** In the following, $m$ is used *a la place de* $j$ for "meet-index".

In [15]:
#   A_ij   B_jk
#     ^^     ^^
#     ||_____||
#     |   |   |
#     |"meet" |
#     | index |
#     |_______|
#         |
#        C_ik

In [16]:
# MAPPER
def matrix_multiply_mapper(m, element):
    # m: the meet-index.
    # element: (matrix-name, i, j, value).
    name, i, j, value = element
    if name=="A":
        for k in range(m): # cuz A_ij is relevant for computing C_ik, k=1,..,m.
            yield((i,k),(j,value)) # using A's col dimension for meet-indexing.
    else:
        for k in range(m):
            yield((k,j),(i,value)) # using B's row dimension for meet-indexing.

In [17]:
# REDUCER
def matrix_multiply_reducer(m, key, indexedValues):
    #                           ^         ^
    #                           |         |
    #   expect C's index (e.g. C_ik)    expect (meet-index, value)
    resultsByIndex = defaultdict(list)
    for index,value in indexedValues:
        resultsByIndex[index].append(value)
    sumProduct = sum(results[0]*results[1] 
                     for results in resultsByIndex.values() # results: [valFromA,valueFromB].
                     if len(results)==2) # non-meet C_ik cells won't have two values appended.
    if sumProduct!=0.0:
        yield (key, sumProduct) # key: C's index (e.g. C_ik); sumProduct: C_ik's values.

In [18]:
# TEST ON TOY EXAMPLE
entries = [("A", 0, 0, 3), ("A", 0, 1, 2), ("B", 0, 0, 4), ("B", 0, 1, -1), ("B", 1, 0, 10)]
mapper = partial(matrix_multiply_mapper, 3)
reducer = partial(matrix_multiply_reducer, 3)
map_reduce(entries, mapper, reducer)
    # output in matrix rep.:
    # [[32,-3, 0],
    #  [ 0, 0, 0]]

[((0, 1), -3), ((0, 0), 32)]

### C. Comparing Brute-Force Matrix Multiplication with MapReduce Version

In [19]:
# MATRIX CONVERTER
#  [[..],[..],..] => [element0, element1, ..].
def matrix_convert(M, matrixName=None):
    if matrixName==None: # randomly generate matrix name: len5 alphanumeric.
        matrixName=''.join(random.choice(string.ascii_uppercase + string.digits) for _ in range(5))
    return [(matrixName, i, j, M[i][j]) for i in range(M.shape[0]) for j in range(M.shape[1]) if M[i][j]!=0]

In [20]:
# SPARSE MATRIX GENERATOR
def sparse_matrix(shape, prop=.1): # prop: proportion of 1's in distribution.
    assert len(shape)==2
    distr = list(np.repeat(0,int((1-prop)*10))) + list(np.repeat(1,int(prop*10)))
    matrix = np.zeros(shape)
    for i in range(shape[0]):
        for j in range(shape[1]):
            matrix[i][j] = random.choice(distr)
    return matrix

In [23]:
M = sparse_matrix((10,10))
matrix_convert(M,matrixName='M')

[('M', 0, 9, 1.0),
 ('M', 1, 5, 1.0),
 ('M', 1, 6, 1.0),
 ('M', 1, 8, 1.0),
 ('M', 2, 7, 1.0),
 ('M', 3, 4, 1.0),
 ('M', 3, 5, 1.0),
 ('M', 4, 0, 1.0),
 ('M', 5, 7, 1.0),
 ('M', 5, 9, 1.0),
 ('M', 6, 6, 1.0),
 ('M', 7, 2, 1.0),
 ('M', 8, 0, 1.0),
 ('M', 9, 1, 1.0),
 ('M', 9, 2, 1.0),
 ('M', 9, 5, 1.0)]

In [24]:
# BRUTE-FORCE MATRIX MULTIPLICATION
def matrix_multiply(A,B):
    assert type(A)==type(B)==np.ndarray # type checking.
    assert A.shape[1]==B.shape[0] # dimension checking.
    def sum_of_product(v,w):
        assert len(v)==len(w)
        total = 0
        for i in range(len(v)):
            total += v[i]*w[i]
        return total
    C = np.zeros((A.shape[0],B.shape[1]))
    for i in range(A.shape[0]):
        for j in range(B.shape[1]):
            C[i][j] = sum_of_product(A[i],B[:,j])
    return C

In [33]:
%%time
A = sparse_matrix((100,500))
B = sparse_matrix((500,100))

CPU times: user 77.5 ms, sys: 2.26 ms, total: 79.8 ms
Wall time: 78.1 ms


In [34]:
%%time
matrix_multiply(A,B)

CPU times: user 1.62 s, sys: 6.25 ms, total: 1.63 s
Wall time: 1.63 s


array([[ 4.,  5.,  6., ...,  7.,  6.,  9.],
       [ 5.,  4.,  2., ...,  4.,  5.,  9.],
       [ 5.,  5.,  7., ...,  7.,  8.,  5.],
       ..., 
       [ 4.,  5.,  2., ...,  9.,  2.,  2.],
       [ 4.,  7.,  3., ...,  3.,  2.,  6.],
       [ 3.,  5.,  4., ...,  4.,  2.,  0.]])

In [35]:
AConv = matrix_convert(A,matrixName='A')
BConv = matrix_convert(B,matrixName='B')
ABList = AConv+BConv
mapper = partial(matrix_multiply_mapper, 500)
reducer = partial(matrix_multiply_reducer, 500)

In [36]:
%%time
map_reduce(ABList, mapper, reducer)
print "DONE"

DONE
CPU times: user 6.57 s, sys: 160 ms, total: 6.73 s
Wall time: 6.71 s


# Application: Building Cooccurrence & Similarity Matrix

In [2]:
import nltk
from nltk.corpus import stopwords
stopwords = stopwords.words('english')
from collections import defaultdict
from functools import partial
from itertools import permutations, product
from string import punctuation

### A. Load & Clean Data

In [3]:
def load_wiki(cutoffFreq=50):
    
    print "... extracting data"
    with open('/Users/jacobsw/Desktop/IMPLEMENTATION_CAMP/CODE/BASIC_TOPICS/DISTRIBUTIONAL_SEMANTICS/ASSIGNMENT_03/wikicorpus.txt','rb') as f:
        raw = f.readlines()
    raw = [sent.split() for sent in raw if sent.startswith('<c>')] 
        # extract sentences; split sentences into word complexes.
    raw = [[map(partial(str.split, word), '|') for word in sent] for sent in raw] 
        # split word complexes into words.
    
    print "... cleaning data"
    sents = [[word[0][1].lower() for word in sent if len(word[0])>1 
              and word[0][2].startswith('N')
              and word[0][1].lower() not in stopwords
              and word[0][1] not in punctuation] for sent in raw]
        # extract lemmas => complete sents corpus .
    
    print "... building token list and vocabulary"
    tokens = [word for sent in sents for word in sent]
        # type: list of words.
    fdist = nltk.FreqDist(tokens)
    vocab = list(set(tokens))   
    
    print "... saving top %d-frequent in vocabulary" % cutoffFreq
    vocab = [word for word in vocab if fdist[word] >= cutoffFreq]
        # vocab is not returned, because the k-frequent cut latter can change it.
    sents = [[word.decode('utf-8','ignore') for word in sent if word in vocab] for sent in sents]
        # type: list of lists of words.
        
    return sents


In [4]:
import warnings
warnings.filterwarnings("ignore")

In [5]:
sents = load_wiki()

... extracting data
... cleaning data
... building token list and vocabulary
... saving top 50-frequent in vocabulary


### B. Build Cooccurrence Matrix

In [6]:
# MAPREDUCE FOR COOCCURRENCE TALLY
def w2w_mapper(sent):
    for w_i,w_j in product(sent,repeat=2):
        yield (tuple(sorted((w_i,w_j))), 1)
def w2w_reducer(pair, ocrs):
    yield (pair, sum(ocrs))
def w2w_mapreduce(sents):
    collector = defaultdict(list)
    for sent in sents:
        for pair,count in w2w_mapper(sent):
            collector[pair].append(count)
    return [output
            for pair,counts in collector.iteritems()
            for output in w2w_reducer(pair,counts)]

In [7]:
%%time
coocrList = w2w_mapreduce(sents)

CPU times: user 27.3 s, sys: 479 ms, total: 27.8 s
Wall time: 27.8 s


In [8]:
%%time
tokens = [pair for pair,_ in coocrList]
tokens = [word for pair in tokens for word in pair]
vocab = list(set(tokens))
print "Token Size: %d | Vocab Size: %d" % (len(tokens),len(vocab))

Token Size: 5555152 | Vocab Size: 6969
CPU times: user 3.73 s, sys: 67.6 ms, total: 3.8 s
Wall time: 3.8 s


In [9]:
coocrList[:3]

[((u'eugene', u'quality'), 6),
 ((u'california', u'pennant'), 2),
 ((u'jensen', u'people'), 6)]

In [10]:
wordToIndex = {word:vocab.index(word) for word in vocab}
indexToWord = {i:w for w,i in wordToIndex.iteritems()}

In [11]:
# STORE (SPARSE) MATRIX IN COOCCURRENCE-ENTRIES LIST
def sparse_cooccurrence_matrix(coocrList):
    sparse = []
    for (w_i,w_j),count in coocrList:
        sparse += [('sp',wordToIndex[w_i],wordToIndex[w_j],count),
                   ('sp',wordToIndex[w_j],wordToIndex[w_i],count)]
    return sparse    

In [12]:
%%time
sp = sparse_cooccurrence_matrix(coocrList)

CPU times: user 4.31 s, sys: 227 ms, total: 4.53 s
Wall time: 4.52 s


### C. Build Similarity Matrix (Cosine)

In [14]:
def matrix_normalize(sparse):
    # generate ||v|| for each word.
    wNorms = defaultdict(int)
    for _,widx_i,widx_j,count in sparse:
        wNorms[widx_i] += count**2
    wNorms = {widx:np.sqrt(countSumSq) for widx,countSumSq in wNorms.iteritems()}
    # normalize by (i,_) cell dividing ||v_i||.
    for k,(spName,widx_i,widx_j,count) in enumerate(sparse):
        sparse[k] = (spName,widx_i,widx_j,float(count)/wNorms[widx_i])
    return sparse

In [15]:
%%time
spNorm = matrix_normalize(sp)

CPU times: user 3.72 s, sys: 84.8 ms, total: 3.8 s
Wall time: 3.8 s


In [16]:
%%time
# NB: this is for MapReduce method.
spNormT = [('spT',widx_j,widx_i,countNorm) for _,widx_i,widx_j,countNorm in spNorm]

CPU times: user 2.32 s, sys: 186 ms, total: 2.51 s
Wall time: 2.51 s


#### a. Method 1: To Numpy Matrix $\Rightarrow$ Matrix Multiplication

In [18]:
%%time
spFullMatrixNorm = np.zeros((len(vocab),len(vocab)))
for _,widx_i,widx_j,countNorm in spNorm:
    spFullMatrixNorm[widx_i][widx_j] = countNorm

CPU times: user 2.67 s, sys: 152 ms, total: 2.82 s
Wall time: 2.8 s


In [19]:
%%time
cosineSimMatrix = np.dot(spFullMatrixNorm, spFullMatrixNorm.T)

CPU times: user 19.9 s, sys: 133 ms, total: 20 s
Wall time: 3.05 s


In [20]:
def cosine_similarity(w1, w2):
    return cosineSimMatrix[wordToIndex[w1]][wordToIndex[w2]]

In [21]:
cosine_similarity('car','bugatti')

0.35866012820941412

#### b. Method 2: MapReduce (current takes ages w/o multiprocessing)

In [66]:
# MAPREDUCE FOR MATRIX MULTIPLICATION
# MAPPER
def matrix_multiply_mapper(m, element):
    # m: the meet-index.
    # element: (matrix-name, i, j, value).
    name, i, j, value = element
    if name=="A":
        for k in range(m): # cuz A_ij is relevant for computing C_ik, k=1,..,m.
            yield((i,k),(j,value)) # using A's col dimension for meet-indexing.
    else:
        for k in range(m):
            yield((k,j),(i,value)) # using B's row dimension for meet-indexing.
# REDUCER
def matrix_multiply_reducer(m, key, indexedValues):
    #                           ^         ^
    #                           |         |
    #   expect C's index (e.g. C_ik)    expect (meet-index, value)
    resultsByIndex = defaultdict(list)
    for index,value in indexedValues:
        resultsByIndex[index].append(value)
    sumProduct = sum(results[0]*results[1] 
                     for results in resultsByIndex.values() # results: [valFromA,valueFromB].
                     if len(results)==2) # non-meet C_ik cells won't have two values appended.
    if sumProduct!=0.0:
        yield (key, sumProduct) # key: C's index (e.g. C_ik); sumProduct: C_ik's values.
# GENERALIZED MAPREDUCE
def map_reduce(inputs, mapper, reducer):
    collector = defaultdict(list)
    for inpt in inputs:
        for key,value in mapper(inpt):
            collector[key].append(value)
    return [output
            for key,values in collector.iteritems()
            for output in reducer(key,values)]

# USAGE
# entries = [("A", 0, 0, 3), ("A", 0, 1, 2), ("B", 0, 0, 4), ("B", 0, 1, -1), ("B", 1, 0, 10)]
# mapper = partial(matrix_multiply_mapper, 3)
# reducer = partial(matrix_multiply_reducer, 3)
# map_reduce(entries, mapper, reducer)
#    output: [((0, 1), -3), ((0, 0), 32)]
#    output in matrix rep.:
#    [[32,-3, 0],
#     [ 0, 0, 0]]

In [ ]:
%%time
pool2matrices = spNorm + spNormT
mapper = partial(matrix_multiply_mapper, len(vocab)) # because the matrix is w2w.
reducer = partial(matrix_multiply_reducer, len(vocab)) 
cosineSimMatrix = map_reduce(pool2matrices, mapper, reducer)

### D. K Most-Similar Words

In [22]:
words = ['car','bus','hospital','hotel','gun','bomb','horse','fox','table','bowl','guitar','piano']

In [27]:
# K Most-Similar Retriever for Numpy Matrix Multiplication method
from operator import itemgetter
def k_most_similar(word, k=10):
    cooccurSims = sorted([(otherWord,cosineSimMatrix[wordToIndex[word]][wordToIndex[otherWord]]) 
                          for otherWord in vocab], key=itemgetter(1), reverse=True)
    return cooccurSims[1:k+1] # excluding word itself, which always has 1.

In [30]:
k_most_similar('car')

[(u'brabham', 0.41523711928675255),
 (u'racing', 0.38803851201317202),
 (u'bugatti', 0.35866012820941412),
 (u'chassis', 0.27478084309186446),
 (u'cable', 0.27377935165510164),
 (u'motors', 0.26590717363846378),
 (u'aston', 0.26406810403387193),
 (u'brake', 0.25814992226216038),
 (u'audi', 0.23974142859235131),
 (u'motor', 0.23868579179851168)]

In [ ]:
# TODO: MULTIPROCESS W/ MAPREDUCE